In [ ]:
import os
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling, BitsAndBytesConfig
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType

# 데이터 로드 및 전처리
df = pd.read_csv('data.csv')
df = df.dropna(subset=['text_column'])  # 텍스트가 없는 행 제거

# 데이터셋 나누기
train_df = df.sample(frac=0.8, random_state=42)
eval_df = df.drop(train_df.index)

# 데이터셋 변환
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)

# 전처리 함수
def preprocess_function(examples, tokenizer):
    return tokenizer(examples['text_column'], truncation=True, padding="max_length", max_length=128)

# 모델과 토크나이저 로드
model_name = "nlpai-lab/KULLM3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=BitsAndBytesConfig(load_in_8bit=True),
    device_map="auto"
)

# Lora 설정
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"]
)
model = get_peft_model(model, peft_config)

# 데이터 전처리
train_dataset = train_dataset.map(lambda x: preprocess_function(x, tokenizer), batched=True)
eval_dataset = eval_dataset.map(lambda x: preprocess_function(x, tokenizer), batched=True)

# 데이터 Collator 설정
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 훈련 인자 설정
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
    save_total_limit=2,
)

# 트레이너 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
)

# 모델 훈련
trainer.train()

# 모델 저장
model.save_pretrained("./model")
tokenizer.save_pretrained("./model")

# 캐시 비우기
torch.cuda.empty_cache()

# 1. 라이브러리 설명
- os:
설명: 운영체제와 상호작용하는 다양한 기능을 제공하는 표준 라이브러리입니다.
사용 이유: 코드에서 파일 경로 설정이나 디렉토리 관리 등에 사용될 수 있습니다. 현재 코드에서는 사용되지 않았지만, 파일 경로를 관리할 필요가 있을 때 유용합니다.
- torch:
설명: 파이토치(PyTorch)는 머신 러닝을 위한 오픈 소스 딥러닝 라이브러리입니다. 주로 텐서 조작 및 모델 훈련에 사용됩니다.
사용 이유: GPU 연산 가속, 텐서 연산, 모델 학습, CUDA 관리 등에 사용됩니다.
- pandas:
설명: 데이터 분석을 위한 고성능 라이브러리로, 특히 데이터 프레임 형식의 데이터 조작에 뛰어납니다.
사용 이유: CSV 파일 로드 및 데이터프레임 조작, 결측값 처리, 데이터 전처리에 사용됩니다.
- transformers:
설명: Hugging Face에서 제공하는 트랜스포머 모델을 위한 라이브러리로, 다양한 자연어 처리 모델과 사전 학습된 모델을 쉽게 사용할 수 있게 합니다.
사용 이유: 사전 학습된 언어 모델(AutoModelForCausalLM), 토크나이저(AutoTokenizer), 트레이너(Trainer), 훈련 인자(TrainingArguments) 등을 관리합니다.
- datasets:
설명: Hugging Face에서 제공하는 데이터셋 처리 라이브러리로, NLP 작업에서 대규모 데이터셋을 관리하고 처리할 수 있습니다.
사용 이유: 파이썬 데이터프레임을 학습용 데이터셋으로 변환하고, 데이터셋을 전처리하고 모델에 입력으로 전달합니다.
- peft:
설명: Parameter Efficient Fine-Tuning (PEFT) 모델을 위한 라이브러리로, 경량화된 모델을 학습하는데 사용됩니다.
사용 이유: LoRA (Low-Rank Adaptation) 설정을 통해 트랜스포머 모델의 특정 모듈만을 경량화하여 학습합니다.
- tqdm:
설명: 반복문에 대한 진행률을 시각적으로 표시해주는 라이브러리입니다.
사용 이유: 데이터 처리나 모델 학습 중 진행 상황을 쉽게 확인할 수 있도록 합니다.

# 2. 함수 및 클래스 설명
- pd.read_csv(data_path):
설명: CSV 파일을 읽어와 pandas 데이터프레임으로 변환합니다.
사용 이유: 학습에 사용할 데이터를 로드하는 데 사용됩니다.
- df.dropna(subset=['text_column']):
설명: 데이터프레임에서 특정 열에 결측값이 있는 행을 제거합니다.
사용 이유: 텍스트 데이터가 없는 행을 제거하여 학습에 사용될 수 있는 유효한 데이터를 확보합니다.
- Dataset.from_pandas(df):
설명: pandas 데이터프레임을 Hugging Face datasets 객체로 변환합니다.
사용 이유: 데이터프레임을 모델 학습에 사용할 수 있도록 데이터셋 형식으로 변환합니다.
- AutoTokenizer.from_pretrained(model_name):
설명: 사전 학습된 토크나이저를 로드합니다. 이 토크나이저는 텍스트 데이터를 모델 입력으로 변환하는 데 사용됩니다.
사용 이유: 텍스트 데이터를 토큰으로 변환하여 모델에 입력으로 제공합니다.
- AutoModelForCausalLM.from_pretrained:
설명: 사전 학습된 언어 모델을 로드합니다. CausalLM은 언어 모델링 작업에 사용됩니다.
사용 이유: 사전 학습된 트랜스포머 기반 언어 모델을 불러와 미세 조정에 사용합니다.
- BitsAndBytesConfig(load_in_8bit=True):
설명: 모델을 8비트 정밀도로 로드하여 메모리 사용을 최적화합니다.
사용 이유: 모델의 메모리 사용량을 줄여서 더 큰 모델을 작은 GPU에서 학습할 수 있도록 합니다.
- get_peft_model:
설명: PEFT 기법을 적용하여 모델의 특정 모듈을 경량화하는 함수입니다.
사용 이유: LoRA 기법을 적용하여 모델의 일부 모듈만 경량화합니다.
- train_dataset.map(lambda x: preprocess_function(x, tokenizer), batched=True):
설명: 데이터셋의 모든 항목에 대해 전처리 함수를 적용합니다.
사용 이유: 토크나이저를 사용하여 텍스트 데이터를 모델이 이해할 수 있는 형태로 변환합니다.
- DataCollatorForLanguageModeling:
설명: 언어 모델 학습을 위한 데이터 배치 생성기를 정의합니다.
사용 이유: 데이터 배치에서 마스킹 처리를 수행하지 않고, 시퀀스 예측을 위한 데이터를 준비합니다.
- TrainingArguments:
설명: 트레이너 객체에 전달되는 하이퍼파라미터와 설정을 정의합니다.
사용 이유: 학습 과정에서 사용할 다양한 설정(학습률, 배치 크기, 에포크 수 등)을 지정합니다.
- Trainer:
설명: 학습 루프를 간소화하는 객체로, 모델 훈련, 평가, 저장을 포함한 모든 작업을 처리합니다.
사용 이유: 미세 조정 과정을 쉽게 구현하고 관리할 수 있습니다.
- trainer.train():
설명: 모델 훈련을 시작하는 메소드입니다.
사용 이유: 설정된 인자와 데이터셋을 바탕으로 모델을 학습합니다.
- model.save_pretrained:
설명: 미세 조정된 모델을 저장합니다.
사용 이유: 학습이 완료된 모델을 저장하여 나중에 사용할 수 있도록 합니다.
- tokenizer.save_pretrained:
설명: 토크나이저를 저장합니다.
사용 이유: 나중에 모델과 함께 사용할 수 있도록 토크나이저를 저장합니다.
- torch.cuda.empty_cache():
설명: 사용하지 않는 CUDA 메모리를 비워 GPU 메모리를 최적화합니다.
사용 이유: 메모리 사용량을 관리하여 더 큰 모델을 학습할 수 있도록 합니다.

# 3. 코드 작동 원리
- 데이터 로드: pandas를 이용해 CSV 파일에서 데이터를 로드합니다. 여기서 text_column은 학습에 사용할 텍스트 데이터가 포함된 열을 나타냅니다.
- 데이터셋 생성: 데이터프레임에서 학습 및 평가용 데이터셋으로 나누고, 이를 Hugging Face의 Dataset 형식으로 변환합니다.
- 모델 및 토크나이저 로드: KULLM3 모델과 토크나이저를 사전 학습된 상태에서 불러옵니다. 모델은 8비트 정밀도로 로드되어 메모리 사용량을 줄입니다.
- PEFT 적용: LoRA 설정을 통해 모델의 특정 모듈을 경량화합니다.
- 데이터 전처리: 텍스트 데이터를 토큰화하고, 이를 모델 입력 형식으로 변환합니다.
- 훈련 설정 및 학습: Trainer 객체를 생성하여 모델 훈련을 수행하고, 학습이 완료되면 모델과 토크나이저를 저장합니다. 마지막으로 캐시를 비워 메모리를 최적화합니다.